In [1]:
!pip install PyTDC
!pip install datasets

!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 39.9 MB/s eta 0:00:00
  Created wheel for PyTDC: filename=PyTDC-0.4.1-py3-none-any.whl size=140644 sha256=30ad08387a081b32cee89fe05107b681823efe8986a34cbc9ec6e3064ed4a211
  Stored in directory: /root/.cache/pip/wheels/14/b7/b8/9d9e1442129743359b0507234ccc0beb67b47173faccd57d54
Successfully built PyTDC


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00


In [2]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
from tdc.multi_pred import DTI
from tdc.generation import MolGen
import plotly.figure_factory as ff

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
data = MolGen(name = 'MOSES')
data=data.get_data()[:100000]

Downloading...
100%|██████████| 75.3M/75.3M [00:04<00:00, 18.1MiB/s]
Loading...
Done!


In [4]:
data=data.sample(frac=1)
data=data.reset_index(drop=True)

## Some Basic Analysis

In [5]:
data['l_smiles']=data.smiles.apply(len)

In [6]:
data['l_smiles'].describe(percentiles=[i/10 for i in range(1,10)])

count    100000.000000
mean         35.150310
std           4.566592
min          15.000000
10%          29.000000
20%          31.000000
30%          33.000000
40%          34.000000
50%          35.000000
60%          36.000000
70%          38.000000
80%          39.000000
90%          41.000000
max          54.000000
Name: l_smiles, dtype: float64

## A solid preset length for drug would be 50

In [7]:
def tokenize(input_string):
  return [ord(char) for char in input_string]
def encode(input_string,max_length=128,padding=True):
  tokens=tokenize(input_string)
  if len(tokens)>max_length:
    tokens=tokens[:max_length]
  if (len(tokens)<max_length) & padding:
    tokens.extend([0 for _ in range(max_length-len(tokens))])
  return tokens
def decode(input_tokens):
  return ''.join(list(map(lambda x:chr(x), input_tokens)))

In [8]:
l_tokenizer=encode('z',padding=False)[0]+1

In [9]:
class Drug_Dataset(Dataset):
    def __init__(self, df,drug_max_length):
        self.df = df
        self.dml=drug_max_length
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        input_drug=torch.tensor(encode(row['smiles'],max_length=self.dml))
        return {'input_drug':input_drug}

In [10]:
dml=50

In [11]:
l=int(data.shape[0]*0.8)
train_p=Drug_Dataset(data[:l],drug_max_length=dml)
test_p=Drug_Dataset(data[l:],drug_max_length=dml)

In [12]:
train_loader=DataLoader(train_p,batch_size=32,shuffle=True)
test_loader=DataLoader(test_p,batch_size=32)

## Let's create the model

## This is Attention, as Torch's attention does not work with mask.

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Encoder(nn.Module):
    def __init__(self, latent_dim=8,embed_dim=16):
        super(Encoder, self).__init__()
        self.latent_dim=latent_dim
        self.embeddings = nn.Embedding(l_tokenizer, embed_dim)
        self.conv1=nn.Conv1d(embed_dim,embed_dim//2,3)
        self.pool1=nn.MaxPool1d(2)
        self.conv2=nn.Conv1d(embed_dim//2,embed_dim//4,3)
        self.pool2=nn.MaxPool1d(2)
        self.conv3=nn.Conv1d(embed_dim//4,embed_dim//8,2)
        self.pool3=nn.MaxPool1d(2)
        self.lsigma=nn.Linear((embed_dim//8)*5, latent_dim)
        self.llogvar=nn.Linear((embed_dim//8)*5, latent_dim)

        # self.N = torch.distributions.Normal(0, 1)
        # self.N.loc = self.N.loc # hack to get sampling on the GPU
        # self.N.scale = self.N.scale
        self.kl = 0
    def reparameterize(self, mu , logvar):
        std = torch.exp(logvar*0.5)
        eps = torch.randn_like(std).to(device)
        z = mu + eps * std
        return z
    def forward(self, x):
        x=self.embeddings(x)
        bn=x.size(0)
        x=torch.transpose(x,1,2)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        # x=torch.transpose(x,1,2)
        x=x.view(bn,-1)
        mu =  self.lsigma(x)
        logvar = self.llogvar(x)
        z=self.reparameterize(mu , logvar)
        self.kl = 0.5*(logvar.exp() + mu**2 - logvar - 1).sum()/x.size(0)/dml/10
        return z

class Decoder(nn.Module):
    def __init__(self, latent_dim=8,dim1=16):
        super(Decoder, self).__init__()
        self.linear=nn.Linear(latent_dim,24)
        self.conv1 = nn.ConvTranspose1d(1, dim1,3,stride=2)
        self.conv2 = nn.ConvTranspose1d(dim1, 2*dim1,2)
        self.linear2 = nn.Linear(2*dim1,l_tokenizer)

    def forward(self, z):
        z = F.relu(self.linear(z))
        z = z.view(-1,1,24)
        z = F.relu(self.conv1(z))
        z = F.relu(self.conv2(z))
        z = torch.transpose(z,1,2)
        z = self.linear2(z)
        return z

class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims=64):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(latent_dims,64).to(device)
        self.decoder = Decoder(latent_dims,32).to(device)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder=VariationalAutoencoder()
opt = torch.optim.Adam(autoencoder.parameters(),lr=0.0005)
loss_fn=torch.nn.CrossEntropyLoss()
def acc_fn(y,y_hat):
  return torch.mean((y==torch.argmax(y_hat,dim=1)).float())
for epoch in range(100):
    train_loss1=0
    train_loss2=0
    train_acc=0
    autoencoder.train()
    for batch in tqdm.tqdm(train_loader):
        x = batch['input_drug'].to(device) # GPU
        opt.zero_grad()
        x_hat = autoencoder(x)
        # x2=torch.transpose(x,1,2)
        x2_hat=torch.transpose(x_hat,1,2)
        loss1=loss_fn(x2_hat,x)
        loss2=autoencoder.encoder.kl
        acc=acc_fn(x,x2_hat)
        loss = loss1 + loss2
        train_acc+=acc.detach().cpu().numpy()
        train_loss1+=loss1.detach().cpu().numpy()
        train_loss2+=loss2.detach().cpu().numpy()
        loss.backward()
        opt.step()

    test_loss1=0
    test_loss2=0
    test_acc=0
    autoencoder.eval()
    with torch.no_grad():
      for batch in tqdm.tqdm(test_loader):
          x = batch['input_drug'].to(device) # GPU
          x_hat = autoencoder(x)
          # x2=torch.transpose(x,1,2)
          x2_hat=torch.transpose(x_hat,1,2)
          loss1=loss_fn(x2_hat,x)
          loss2=autoencoder.encoder.kl
          acc=acc_fn(x,x2_hat)
          loss = loss1 + loss2
          test_acc+=acc.detach().cpu().numpy()
          test_loss1+=loss1.detach().cpu().numpy()
          test_loss2+=loss2.detach().cpu().numpy()

    print(f"TRAIN: EPOCH {epoch}: SSE: {train_loss1/len(train_loader)}, KL_LOSS: {train_loss2/len(train_loader)}, ACC: {train_acc/len(train_loader)}   \nTEST: EPOCH {epoch}: SSE: {test_loss1/len(test_loader)}, KL_LOSS: {test_loss2/len(test_loader)}, ACC: {test_acc/len(test_loader)}")

100%|██████████| 625/625 [00:03<00:00, 175.79it/s]


TRAIN: EPOCH 0: SSE: 1.7329068256378173, KL_LOSS: 0.04536705662759487, ACC: 0.4831164893992711   
TEST: EPOCH 0: SSE: 1.557160227203369, KL_LOSS: 0.04142424785494805, ACC: 0.521051988697052


100%|██████████| 625/625 [00:03<00:00, 176.78it/s]


TRAIN: EPOCH 1: SSE: 1.4094014980316163, KL_LOSS: 0.06349502699226141, ACC: 0.5593669870257377   
TEST: EPOCH 1: SSE: 1.2341641063690185, KL_LOSS: 0.08825779937505722, ACC: 0.6001389857292175


100%|██████████| 625/625 [00:03<00:00, 175.64it/s]


...

TRAIN: EPOCH 97: SSE: 0.5986001903891564, KL_LOSS: 0.11718855932354927, ACC: 0.813389230465889   
TEST: EPOCH 97: SSE: 0.6020316452026367, KL_LOSS: 0.1171969602227211, ACC: 0.8120879807472229


100%|██████████| 625/625 [00:04<00:00, 154.84it/s]


TRAIN: EPOCH 98: SSE: 0.5966527721047401, KL_LOSS: 0.11717203666865826, ACC: 0.8136819811344147   
TEST: EPOCH 98: SSE: 0.590332763671875, KL_LOSS: 0.11704137345552444, ACC: 0.8149979801177979


100%|██████████| 625/625 [00:04<00:00, 130.89it/s]

TRAIN: EPOCH 99: SSE: 0.5967556195020676, KL_LOSS: 0.11728634235560895, ACC: 0.8137077312707901   
TEST: EPOCH 99: SSE: 0.6018684244155884, KL_LOSS: 0.11682128728628159, ACC: 0.8115099796295167


In [39]:
x_hat[0].argmax(-1)

tensor([67, 99, 49, 99, 99, 40, 67, 41, 99, 99, 40, 45, 99, 49, 99, 99, 40, 45,
        99, 50, 99, 99, 99, 40, 99, 41, 99, 99, 99, 41, 99, 99, 99, 41, 40, 50,
        41, 99, 49,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       device='cuda:0')

In [41]:
x[0]

tensor([ 67,  99,  49,  99,  99,  40,  67,  41, 110,  99,  40,  78,  99,  50,
        110,  99,  40,  45,  99,  51,  99,  99,  99,  40,  78,  41,  99,  99,
         51,  41, 110,  91, 110,  72,  93,  50,  41, 110,  49,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0], device='cuda:0')